<a href="https://colab.research.google.com/github/fanyang2012/Grid-Studio/blob/master/Master_thesis_Chapter_3_Exploratory_data_analysis_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data understanding

## 1.1 Import dataset 'movedata' for five years

In [ ]:
import numpy as np
import ray
import os
import datetime as dt
ray.init(num_cpus=20, ignore_reinit_error=True) 
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as pd

In [ ]:
movedata_path = "/media/fs/ML_Projects/technilog/exports/AIM_TNL_003_001/bewegungsdaten/bewegungen_historic_export_new_format.csv"
movedata_header = pd.read_csv(movedata_path, sep=';', nrows=1)
movedata1 = pd.read_csv(movedata_path, sep=';', header=None, skiprows=12175634, nrows=8018141)
movedata2 = pd.read_csv(movedata_path, sep=";", header=None, skiprows=20193775, nrows=13447308)
movedata = movedata1.append(movedata2)
movedata.columns = movedata_header.columns

## 1.2 First glance at dataset

In [ ]:
movedata.head()
movedata.info()
movedata.describe()
movedata['skuGroupId'].nunique()

## 1.3 Identify missing values

In [ ]:
novalue = movedata[movedata['skuGroupId'] == ' '].shape[0] # Number of rows where skuGroupId is ' '
navalue = movedata['skuGroupId'].isna().sum() # Number of rows where skuGroupId is na
(novalue + navalue)/ movedata.shape[0]

## 1.4 Data preprocessing
* Rename column names
* Remain date time only date
* Delete missing values

In [ ]:
movedata_copy = movedata.copy() # copy original dataset
movedata_copy.columns = movedata_copy.columns.str.capitalize() # capitalize the column names
movedata_copy["Day"]=pd.to_datetime(movedata_copy["Date"]).dt.date # add a new colume which just shows the Y-M-D
movedata_copy = movedata_copy[movedata_copy['Skugroupid'].notna()] # drop the rows where there are missing values of Skugroupid
movedata_copy = movedata_copy[movedata_copy['Skugroupid'] != ' ']  # drop the rows where there are no values of Skugroupid

In [ ]:
movedata_copy.shape[0] # number of rows in new dataset
movedata_copy['Skugroupid'].nunique() # number of sku groups in new dataset

## 1.5 Generate dataset only with the columns of 'Skugroupid' 'Day' and 'Pick volume' by counting 'Pickid'

In [ ]:
groupdata = movedata_copy.groupby(["Skugroupid", "Day"])["Pickid"].count().reset_index()

In [ ]:
# Have a first overview of 'groupdata'
groupdata.head()
groupdata.tail()
groupdata.info()
groupdata.describe()

## 1.6 Identify Skugroupid where there are only several rows

In [ ]:
groupname = groupdata['Skugroupid'].unique() # list of group name
print('group12:\n\n{}\n'.format(groupdata[groupdata['Skugroupid'] == groupname[12]]))
print('group13:\n\n{}'.format(groupdata[groupdata['Skugroupid'] == groupname[13]]))

## 1.7 Delete 'Skugroupid' where there are only several rows

In [ ]:
group_data = groupdata.copy()
for i in range(len(groupname)):
    if group_data[group_data['Skugroupid'] == groupname[i]].shape[0] < 365:
        group_data = group_data[group_data['Skugroupid'] !=groupname[i]]

In [ ]:
print(group_data.shape[0]) # check the rows of new dataset
print('Number of groups in groupdata:\n\n{}\n'.format(groupdata['Skugroupid'].nunique())) # check the number of sku groups in 'groupdata'
print('Number of groups in group_data:\n\n{}\n'.format(group_data['Skugroupid'].nunique())) # check the number of sku groups in 'group_data'
print('Ratio:\n\n{}'.format((groupdata['Skugroupid'].nunique() - group_data['Skugroupid'].nunique()) / groupdata['Skugroupid'].nunique())) # print the ratio which are dropped

## 1.8 Identify that the date of each 'Skugroupid' in 'group_data'  is not consistent

In [ ]:
group_data.head()
group_data.tail()

## 1.9 Impute the dataset and fill the 'Pick volume' of missing rows with zero

In [ ]:
group_data['Day'] = pd.to_datetime(group_data['Day'])
group_data['Skugroupid'] = group_data['Skugroupid'].astype(int)

# create a DataFrame with full_date in order to merge with groupdata
full_date = pd.DataFrame({'Day': pd.date_range('2015-01-01','2019-12-31')})
groupname = group_data['Skugroupid'].unique() # list of group name

# create a DataFrame without any values
new_groupdata = pd.DataFrame(columns=['Skugroupid', 'Day', 'Pick volume'])

for i in range(len(groupname)):
        # output merged data betw. groupdata with groupname i and full_date
    merged_data = group_data[group_data['Skugroupid'] == groupname[i]].merge(full_date, on = 'Day', how = 'outer').sort_values('Day')
        # impute the missing value with group no.
    merged_data['Skugroupid'].fillna(groupname[i], inplace = True)
        # impute the missing value for pick volume with 0
    merged_data['Pick volume'].fillna(0, inplace = True)
        # append the mergedata into the new_groupdata step by step
    new_groupdata = new_groupdata.append(merged_data, ignore_index=True)

print(new_groupdata)

## 1.10 First glance at the final dataset 'new_groupdata'

In [ ]:
new_groupdata.head() # first five rows
new_groupdata.tail() # last five rows
new_groupdata.info() # data type and shape of each column
new_groupdata.describe() # basic statistical characteristics of each column
new_groupdata['Skugroupid'].nunique() # number of sku groups
new_groupdata.shape # number of rows and columns

## 1.11 Export dataset

In [ ]:
new_groupdata.to_csv('new_groupdata', index = False)

# 2. Exploratory data analysis

## 2.1 Basic statistical characteristics of the dataset

In [ ]:
new_groupdata['Pick volume'].mean() # mean
new_groupdata['Pick volume'].std() # standard deviation
new_groupdata['Pick volume'].median() # median
new_groupdata['Pick volume'].sum() # sum
new_groupdata['Pick volume'].max() # maximum
new_groupdata['Pick volume'].min() # minimum
new_groupdata[new_groupdata['Pick volume']==0]['Skugroupid'].count() / new_groupdata.shape[0] # the percentage of zero value

## 2.2 Mean value of each sku group

In [ ]:
mean_sku = new_groupdata.groupby('Skugroupid')['Pick volume'].mean().sort_values(ascending=False).reset_index()
mean_sku.columns = ['Skugroupid', 'mean']
mean_sku['Skugroupid'] = mean_sku['Skugroupid'].astype(str)

### Overview of the top 100 SKU groups with the highest average pick volumes

In [ ]:
plt.figure(figsize=(20,8))
plt.bar(np.arange(1, 101), mean_sku['mean'][:100], width = 0.8)

plt.xlabel('SKU groups', fontsize = 24)
plt.ylabel('Mean of pick volumes', fontsize = 24)
plt.title('Overview of the top 100 SKU groups with the highest average pick volumes', fontsize = 24)
plt.tick_params(labelsize=24)

plt.show()

## 2.4 Define different levels of mean value in dataset

In [ ]:
def division(data, col): 
    
    data.index = np.arange(data.shape[0])
    
    for j in range(data.shape[0]):
    
        if data.loc[j, col] < 10: 
            data.loc[j, 'division'] = '0-10'
        elif data.loc[j, col] < 20: 
            data.loc[j, 'division'] = '10-20'
        elif data.loc[j, col] < 40:
            data.loc[j, 'division'] = '20-40'
        elif data.loc[j, col] < 60: 
            data.loc[j, 'division'] = '40-60'        
        elif data.loc[j, col] < 80: 
            data.loc[j, 'division'] = '60-80'        
        elif data.loc[j, col] < 100: 
            data.loc[j, 'division'] = '80-100'        
        else: 
            data.loc[j, 'division'] = 'over 100'        
        
    return data

In [ ]:
mean_sku['division'] = 0
mean_sku = division(mean_sku, 'mean').sort_values(by = 'mean', ascending=True)

In [ ]:
division = mean_sku.groupby('division')['Skugroupid'].count().reset_index()

In [ ]:
division.columns = ['division', 'Number of Skugroups']
division['Proportions'] = division['Number of Skugroups'] / 465
division

In [ ]:
plt.figure(figsize=(20,8))

plt.bar(division['division'], division['Number of Skugroups'], width = 0.4, label = 'Number of SKU groups                                   ')
#plt.plot(division['division'], division['Proportions'])
plt.grid()
plt.legend(loc = 1)
plt.ylabel('Bar chart', fontsize = 14)
plt.title('Distribution of average pick volumnes in different tiers', fontsize = 16)

axes2 = plt.twinx()
axes2.plot(division['division'], division['Proportions'], color='r', marker = '*', label='Proportions')
axes2.set_ylabel('Line plot', fontsize = 14)
axes2.legend( loc='upper right')

plt.show()

## 2.5 Examples of sku groups in different tiers

In [ ]:
list = [117050, 211060, 212020, 113010, 421010, 232010]

for i in range(6):
    globals()["ts" + str(i+1)] = new_groupdata[new_groupdata['Skugroupid'] == list[i]]
    globals()["ts" + str(i+1)]['Day'] = pd.to_datetime(globals()["ts" + str(i+1)]['Day'])

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6))  = plt.subplots(3, 2, figsize=(30, 15))

ax1.plot(ts1['Day'], ts1['Pick volume'], color='dimgray', marker='.', linestyle='-', linewidth=0.5)
ax1.set_title('Example of SKU group: Average pick volumes - [0, 20] ', fontsize = 15)
ax2.plot(ts1['Day'], ts2['Pick volume'], color='dimgray', marker='.', linestyle='-', linewidth=0.5)
ax2.set_title('Example of SKU group: Average pick volumes - [20, 40]', fontsize = 15)
ax3.plot(ts1['Day'], ts3['Pick volume'], color='dimgray', marker='.', linestyle='-', linewidth=0.5)
ax3.set_title('Example of SKU group: Average pick volumes - [40, 60]', fontsize = 15)
ax4.plot(ts1['Day'], ts4['Pick volume'], color='dimgray', marker='.', linestyle='-', linewidth=0.5)
ax4.set_title('Example of SKU group: Average pick volumes - [60, 80]', fontsize = 15)
ax5.plot(ts1['Day'], ts5['Pick volume'], color='dimgray', marker='.', linestyle='-', linewidth=0.5)
ax5.set_title('Example of SKU group: Average pick volumes - [80, 100]', fontsize = 15)
ax6.plot(ts1['Day'], ts6['Pick volume'], color='dimgray', marker='.', linestyle='-', linewidth=0.5)
ax6.set_title('Example of SKU group: Average pick volumes - [>= 100]', fontsize = 15)
  
plt.show()

## 2.6 Distribution of average pick volumes in year, month, week, day and weekday

In [ ]:
def date_features(data): 
    
    data['Day'] = pd.to_datetime(data['Day'])
    
    data['Date_Day'] = data['Day'].dt.day
    data['Date_weekday'] = data['Day'].dt.weekday
    data['Date_week'] = data['Day'].dt.isocalendar().week.astype('int64')
    data['Date_month'] = data['Day'].dt.month
    data['Date_year'] = data['Day'].dt.year    
    
    return data

In [ ]:
new_dt = new_groupdata.copy()
new_dt = date_features(new_dt)

### 2.6.1 Distribution of average pick volumes in year

In [ ]:
year_dis = new_dt.groupby(['Skugroupid','Date_year'])['Pick volume'].mean().reset_index()

In [ ]:
plt.figure(figsize=(20,8))
all_year = [year_dis[year_dis['Date_year'] == 2015]['Pick volume'], year_dis[year_dis['Date_year'] == 2016]['Pick volume'], year_dis[year_dis['Date_year'] == 2017]['Pick volume'],
           year_dis[year_dis['Date_year'] == 2018]['Pick volume'],year_dis[year_dis['Date_year'] == 2019]['Pick volume']]

box = plt.boxplot(all_year, showmeans=True, showfliers=False, patch_artist=True, boxprops={'facecolor': 'lightblue'})

plt.xlabel('Year', fontsize = 14)
plt.ylabel('Mean of Pick volumes', fontsize = 14)
plt.title('Distribution of mean of pick volumes in year from 2015 to 2019', fontsize = 16)
    
plt.show()

### 2.6.2 Distribution of average pick volumes in month

In [ ]:
month_dis = new_dt.groupby(['Skugroupid','Date_month'])['Pick volume'].mean().reset_index()
all_month = []
for i in range(1,13):
    globals()["m" + str(i)] = month_dis[month_dis['Date_month'] == i]['Pick volume']
    all_month.append(globals()["m" + str(i)])

In [ ]:
plt.figure(figsize=(20,8))
plt.boxplot(all_month, showfliers = False, showmeans=True, patch_artist=True, boxprops={'facecolor': 'lightblue'})  # fill with color
plt.xlabel('Month', fontsize = 24)
plt.ylabel('Mean of Pick volumes', fontsize = 24)
plt.tick_params(labelsize = 22)
plt.title('Distribution of mean of pick volumes in month', fontsize = 24)
plt.show()

### 2.6.3 Distribution of average pick volumes in week

In [ ]:
week_dis = new_dt.groupby(['Skugroupid','Date_week'])['Pick volume'].mean().reset_index()
all_week = []
for i in range(1,53):
    globals()["w" + str(i)] = month_dis[week_dis['Date_week'] == i]['Pick volume']
    all_week.append(globals()["w" + str(i)])

In [ ]:
plt.figure(figsize=(20,8))
plt.boxplot(all_week, showfliers = False, showmeans=True, patch_artist=True, boxprops={'facecolor': 'lightblue'})  # fill with color
plt.xlabel('Week', fontsize = 14)
plt.ylabel('Mean of Pick volumes', fontsize = 14)
plt.title('Distribution of mean of pick volumes in week', fontsize = 16)
plt.show()

### 2.6.4 Distribution of average pick volumes in day

In [ ]:
day_dis = new_dt.groupby(['Skugroupid','Date_Day'])['Pick volume'].mean().reset_index()
all_day = []
for i in range(1,32):
    globals()["d" + str(i)] = day_dis[day_dis['Date_Day'] == i]['Pick volume']
    all_day.append(globals()["d" + str(i)])

In [ ]:
plt.figure(figsize=(20,8))

plt.boxplot(all_day, showfliers = False, showmeans=True, patch_artist=True, boxprops={'facecolor': 'lightblue'})  # fill with color
plt.xlabel('Day of month', fontsize = 24)
plt.ylabel('Mean of Pick volumes', fontsize = 24)
plt.title('Distribution of mean of pick volumes in day', fontsize = 24)
plt.tick_params(labelsize = 16)

plt.show()

### 2.6.4 Distribution of average pick volumes in weekday

In [ ]:
wd_dis = new_dt.groupby(['Skugroupid','Date_weekday'])['Pick volume'].mean().reset_index()

In [ ]:
all_wd = []
for i in range(1,7):
    globals()["d" + str(i)] = wd_dis[wd_dis['Date_weekday'] == i]['Pick volume']
    all_wd.append(globals()["d" + str(i)])

In [ ]:
plt.figure(figsize=(20,8))

plt.boxplot(all_wd, showfliers = False, showmeans=True, patch_artist=True, boxprops={'facecolor': 'lightblue'})  # fill with color
plt.xlabel('Weekday', fontsize = 14)
plt.ylabel('Mean of Pick volumes', fontsize = 14)
plt.title('Distribution of mean of pick volumes in weekday', fontsize = 16)

plt.show()